1. на основе word2vec/fasttext реализовать метод поиска ближайших статей
(на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы


In [1]:
import pandas as pd
import numpy as np
import re
import string


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


import os
import re
from string import punctuation


from nltk.stem.snowball import SnowballStemmer

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText

In [2]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [3]:
# пример работы с ним 
from corus import load_lenta
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [4]:
df = pd.read_csv('lenta-ru-news.csv.gz', compression='gzip', float_precision="high")

In [5]:
df

,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/14/cancer/,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия,Общество
1,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды
2,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир
3,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика
4,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество
...,...,...,...,...,...
739346,https://lenta.ru/news/1999/08/31/cholera/,Южно-Сахалинск объявлен очагом холеры,Сегодня областной центр Сахалина и Курил получ...,Россия,Все
739347,https://lenta.ru/news/1999/08/31/polit_blok/,Леворадикалы создают предвыборный блок,Бывший шеф Службы безопасности президента Але...,Россия,Все
739348,https://lenta.ru/news/1999/08/31/dagestan/,В горах Дагестана идут активные боевые действия,Сегодня утром в районах дагестанских селений Ч...,Россия,Все
739349,https://lenta.ru/news/1999/08/31/karachaevo/,Карачаево-Черкесия раскололась по национальном...,Намеченная на сегодняшний день церемония вступ...,Россия,Все


In [6]:
stop_words = stopwords.words('russian')
stemmer = SnowballStemmer(language='russian')

In [7]:
from nltk import sent_tokenize, word_tokenize

In [8]:
patterns = '[A-Za-z0-9!@❤є"“’«»#$%&\'()*+,—/:;<=>?^_`{|}~\[\]]'
pattern_url = 'http[s]?://\S+|www\.\S+'
pattern_tags = '<.*?>'
morpher = MorphAnalyzer()


def transformer(text):
    text = text.lower()
    text = re.sub(patterns, ' ', text)
    text = re.sub(pattern_url, ' ', text)
    text = re.sub(pattern_tags, ' ', text)
    text = re.sub(r'[^\w\s\n]', ' ', text)
    text = re.sub(r'ð', ' ', text)
    filtered_tokens = []
    tokens = [word for sent in sent_tokenize(text) for word in word_tokenize(sent)]
    for token in tokens:
        if token not in stop_words and token != " " and token.strip() not in punctuation:
            filtered_tokens.append(token)
    tokens_word = [t for t in filtered_tokens if t not in stop_words]
    #tokens_word = [morpher.parse(i)[0].normal_form for i in tokens_word]
    
    new_text = ' '.join(tokens_word)
    

    return new_text

In [9]:
df[['text', 'title']] = df[['text', 'title']].astype(str)

In [10]:
df['text'] = df['text'].apply(transformer)

In [11]:
df['title'] = df['title'].apply(transformer)

In [12]:
df.head(5)

,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/14/cancer/,названы регионы россии самой высокой смертност...,вице премьер социальным вопросам татьяна голик...,Россия,Общество
1,https://lenta.ru/news/2018/12/15/doping/,австрия представила доказательств вины российс...,австрийские правоохранительные органы представ...,Спорт,Зимние виды
2,https://lenta.ru/news/2018/12/15/disneyland/,обнаружено самое счастливое место планете,сотрудники социальной сети проанализировали по...,Путешествия,Мир
3,https://lenta.ru/news/2018/12/15/usa25/,сша раскрыли сумму расходов расследование росс...,начала расследования российского вмешательства...,Мир,Политика
4,https://lenta.ru/news/2018/12/15/integrity/,хакеры рассказали планах великобритании замини...,хакерская группировка опубликовала новые докум...,Мир,Общество


In [13]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

for line in df['text']:
    spls = transformer(line)
    sentences.append(spls)

In [14]:
sentences = [i for i in sentences if len(i) > 2]

In [15]:
sentences[0]

'вице премьер социальным вопросам татьяна голикова рассказала каких регионах россии зафиксирована наиболее высокая смертность рака сообщает риа новости словам голиковой чаще онкологические заболевания становились причиной смерти псковской тверской тульской орловской областях также севастополе вице премьер напомнила главные факторы смертности россии рак болезни системы кровообращения начале года стало известно смертность онкологических заболеваний среди россиян снизилась впервые года данным росстата году рака умерли тысяч человек это процента меньше годом ранее'

In [16]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=3)
modelFT = FastText(sentences=sentences, vector_size=300, min_count=3, window=5, workers=8)

In [17]:
import annoy

In [18]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0


for line in df['text']:
    n_w2v = 0
    n_ft = 0

    index_map[counter] = line
    question = transformer(line)

    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in question:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)

    counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
ft_index.build(10)

True

In [19]:
def get_response(question, index, model, index_map):
    question = transformer(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [39]:
TEXT = " Санкции США против России"

In [41]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['дисциплинарный комитет международной федерации футбола фифа оштрафовал российский футбольный союз рфс расистские выкрики болельщиков время товарищеского матча сборными россии франции сообщается сайте организации информации пресс службы сумма штрафа наложенного рфс составит тысяч швейцарских франков тысяч долларов федерации отметили расистские кричалки позволили лишь несколько россиян матч сборными россии франции состоялся санкт петербурге марта встреча завершилась победой французов счетом мячей забил поль погба единственным мячом составе сборной россии отметился нападающий федор смолов матча фотограф заявил российские болельщики позволили расистские выкрики адрес французских футболистов погба усмана дембеле',
 'компания сменила роли технического спонсора олимпийского комитета россии окр говорится сообщении организации опубликованном странице окр компания официальным экипировщиком олимпийской команды россии решение итогам проведенного конкурса утвердил исполком окр говорится сообщении

In [37]:
get_response(TEXT, ft_index, modelFT, index_map)

['сша скончалась американская сценаристка глория кац четверг ноября сообщает лет рассказал муж кац режиссер сценарист уиллард хайк умерла воскресенье ю годовщину свадьбы больнице лос анджелеса кац боролась раком яичников фильмографии глории кац картины американские граффити индиана джонс храм судьбы говард утка кац мужем также тайно работали сценарием звездным войнам',
 'нападающие шанхай сипг бразилец халк китаец ву лэй также главный тренер команды португалец андре виллаш боаш дисквалифицированы тура чемпионата китая поддержку бразильского полузащитника оскара сообщается сайте китайской футбольной ассоциации кфа бывший хавбек челси оскар выступающий шанхай дисквалифицирован восемь матчей чемпионата страны инициирование массовой драки время встречи чемпионата страны против гуанчжоу фули дисквалификации оскара халк ву лэй сфотографировались футболках надписью ставшей неофициальным слоганом поддержку делать говорить португальский специалист виллаш боаш также публично высказывал недовольс